# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
#from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=128, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=128, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F



class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out=self.conv2(out)
        out = F.relu(self.bn2(out*torch.randn(out.shape).cuda()*0.1))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            out_planes = x if isinstance(x, int) else x[0]
            stride = 1 if isinstance(x, int) else x[1]
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
if torch.cuda.is_available():
    model=MobileNet(10).cuda()
else:
    model=MobileNet(10)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            if batch_idx % 4*show_step==0:
                print(torch.abs(model.layers[12].conv2.weight).sum())
                print(model.layers[12].conv2.weight.shape)
                print(model.layers[12].conv2.weight)
                
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [8]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [ ]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

#---------------------------- Test ------------------------------
test(epoch)

tensor(17993.4043, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[-9.6153e-03]],

         [[-1.7218e-02]],

         [[-2.2505e-02]],

         ...,

         [[-1.5606e-02]],

         [[ 1.0577e-02]],

         [[-1.2389e-02]]],


        [[[-1.7796e-02]],

         [[ 1.1535e-02]],

         [[ 3.4615e-02]],

         ...,

         [[-1.2994e-03]],

         [[ 1.3730e-02]],

         [[ 2.6112e-02]]],


        [[[ 1.6887e-02]],

         [[-1.3684e-02]],

         [[-1.4252e-02]],

         ...,

         [[ 1.2591e-02]],

         [[-3.2445e-04]],

         [[-1.7483e-02]]],


        ...,


        [[[-1.8046e-02]],

         [[ 3.5776e-02]],

         [[ 3.0765e-02]],

         ...,

         [[ 3.0758e-02]],

         [[ 3.6212e-02]],

         [[ 9.4577e-03]]],


        [[[ 2.0593e-02]],

         [[-2.0386e-02]],

         [[ 2.8472e-02]],

         ...,

         [[ 5.9590e-03]],

         [[ 4.0105e-02]],

         [[-1.5472e-02]]],


 

Train Epoch: 0 [16640/50000 (37%)]	Loss: 1.969598, Accuracy: 22.66
tensor(71490.3906, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 8.9012e-02]],

         [[-2.9345e-02]],

         [[ 8.5338e-02]],

         ...,

         [[ 7.6778e-02]],

         [[ 1.2487e-02]],

         [[-3.6156e-02]]],


        [[[-3.8664e-02]],

         [[-3.2099e-02]],

         [[ 1.0460e-01]],

         ...,

         [[-4.6817e-03]],

         [[ 5.8476e-02]],

         [[ 1.0689e-01]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 2.3976e-02]],

         [[ 6.3463e-02]],

         [[ 1.0973e-01]],

         ...,

         [[ 5.5833e-02]],

         [[ 6.7959e-02]],

         [[ 6.0602e-02]]],


        [[[-8.3039e-02]],

         [[-4.7637e-03]],

         [[ 5.1115e-02]],

         ...,

         [[ 1.562

Train Epoch: 0 [34560/50000 (77%)]	Loss: 2.008219, Accuracy: 18.75
tensor(81514.3828, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.1518e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[ 8.8833e-02]],

         [[ 1.3930e-02]],

         [[-6.4006e-02]]],


        [[[ 2.7456e-03]],

         [[-3.2099e-02]],

         [[ 1.0370e-01]],

         ...,

         [[-1.0677e-02]],

         [[ 3.2976e-02]],

         [[ 8.5551e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 9.9072e-03]],

         [[ 6.3463e-02]],

         [[ 1.0367e-01]],

         ...,

         [[ 1.2416e-01]],

         [[ 9.8655e-02]],

         [[ 3.2623e-02]]],


        [[[-7.5704e-02]],

         [[-4.7640e-03]],

         [[ 4.9915e-02]],

         ...,

         [[-5.562

Train Epoch: 1 [6400/50000 (14%)]	Loss: 1.937477, Accuracy: 21.88
tensor(87174.5859, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.1624e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[ 9.0330e-02]],

         [[ 1.3885e-02]],

         [[-6.2873e-02]]],


        [[[ 6.5778e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-9.1497e-03]],

         [[ 2.7887e-02]],

         [[ 8.0902e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 4.7829e-02]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 1.3637e-01]],

         [[ 6.6089e-02]],

         [[ 4.9019e-02]]],


        [[[-1.4197e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,

         [[-1.0459

Train Epoch: 1 [24320/50000 (54%)]	Loss: 2.014601, Accuracy: 25.00
tensor(91859.6016, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.1624e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[ 9.0330e-02]],

         [[ 1.3885e-02]],

         [[-6.2873e-02]]],


        [[[ 8.7044e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-9.5347e-03]],

         [[ 2.7205e-02]],

         [[ 7.8325e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.0413e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 1.5834e-01]],

         [[ 1.0303e-01]],

         [[ 4.1356e-02]]],


        [[[-1.3090e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,

         [[-4.302

...

Train Epoch: 20 [16640/50000 (37%)]	Loss: 1.705610, Accuracy: 34.38
tensor(1.4180e+05, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.0585e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[-7.3888e-02]],

         [[ 2.7258e-01]],

         [[-2.6770e-01]]],


        [[[ 8.9786e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-6.0769e-03]],

         [[ 2.5716e-02]],

         [[ 7.7795e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.1236e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 1.9901e-01]],

         [[ 1.0061e-01]],

         [[ 2.2357e-02]]],


        [[[-1.7011e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,

         [[-3.94

Train Epoch: 20 [34560/50000 (77%)]	Loss: 1.644211, Accuracy: 36.72
tensor(1.00000e+05 *
       1.4210, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.0585e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[-7.3888e-02]],

         [[ 2.7258e-01]],

         [[-2.6770e-01]]],


        [[[ 8.9786e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-6.0769e-03]],

         [[ 2.5716e-02]],

         [[ 7.7795e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.1236e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 1.9900e-01]],

         [[ 1.0061e-01]],

         [[ 2.2353e-02]]],


        [[[-1.7011e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,


tensor(1.00000e+05 *
       1.4225, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.0585e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[-7.3888e-02]],

         [[ 2.7258e-01]],

         [[-2.6770e-01]]],


        [[[ 8.9786e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-6.0769e-03]],

         [[ 2.5716e-02]],

         [[ 7.7795e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.1236e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 2.0178e-01]],

         [[ 9.9639e-02]],

         [[ 2.2425e-02]]],


        [[[-1.7011e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,

         [[-3.9494e-02]],

         [[-1.2667e-03]],

         [[ 3

Train Epoch: 21 [21760/50000 (48%)]	Loss: 1.608602, Accuracy: 37.50
tensor(1.00000e+05 *
       1.4234, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.0585e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[-7.3888e-02]],

         [[ 2.7258e-01]],

         [[-2.6770e-01]]],


        [[[ 8.9786e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-6.0769e-03]],

         [[ 2.5716e-02]],

         [[ 7.7795e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.1236e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 2.0246e-01]],

         [[ 9.9423e-02]],

         [[ 2.2438e-02]]],


        [[[-1.7011e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,


Train Epoch: 21 [39680/50000 (88%)]	Loss: 1.588836, Accuracy: 35.94
tensor(1.00000e+05 *
       1.4243, device='cuda:0')
torch.Size([1024, 1024, 1, 1])
Parameter containing:
tensor([[[[ 1.0585e-01]],

         [[-2.9345e-02]],

         [[ 8.4756e-02]],

         ...,

         [[-7.3888e-02]],

         [[ 2.7258e-01]],

         [[-2.6770e-01]]],


        [[[ 8.9786e-03]],

         [[-3.2099e-02]],

         [[ 1.0369e-01]],

         ...,

         [[-6.0769e-03]],

         [[ 2.5716e-02]],

         [[ 7.7795e-02]]],


        [[[ 5.7642e-02]],

         [[-2.0936e-02]],

         [[-9.6480e-03]],

         ...,

         [[ 1.1144e-01]],

         [[-3.2234e-02]],

         [[ 4.9874e-02]]],


        ...,


        [[[ 1.1236e-01]],

         [[ 6.3463e-02]],

         [[ 1.0353e-01]],

         ...,

         [[ 2.0246e-01]],

         [[ 9.9423e-02]],

         [[ 2.2438e-02]]],


        [[[-1.7011e-01]],

         [[-4.7640e-03]],

         [[ 4.9849e-02]],

         ...,


# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

